# Estimativas de Localização de População e Taxas de Homicídio

## Calcular a média, a média aparada e a mediana para a população

In [16]:
import pandas as pd
import numpy as np

from scipy import stats

In [2]:
state = pd.read_csv('state.csv')

In [13]:
state.shape

(50, 4)

In [35]:
state.head(8)

,State,Population,MurderRate,Abbreviation
49,Wyoming,563626,2.7,WY
44,Vermont,625741,1.6,VT
33,North Dakota,672591,3.0,ND
1,Alaska,710231,5.6,AK
40,South Dakota,814180,2.3,SD
7,Delaware,897934,5.8,DE
25,Montana,989415,3.6,MT
38,Rhode Island,1052567,2.4,RI


In [31]:
state = state.rename(columns={'Murder.Rate': 'MurderRate'}) # Rename para retirar o ponto '.' da coluna

In [36]:
# média
state['Population'].mean()

6162876.3

In [37]:
# média parada
stats.trim_mean(state['Population'], 0.1) # A função trim_mean exclui % de cada ponta - os cinco maiores e menores estados

4783697.125

In [38]:
# mediana
state['Population'].median()

4436369.5

## A média é maior que a média aparada, que é maior que a mediana. 

## Calcular a taxa média de homicídios para o país. Necessário calcular a média ou mediana ponderada

In [33]:
# média ponderada
np.average(state["MurderRate"], weights=state["Population"])

4.445833981123393

In [49]:
# mediana ponderada
state.sort_values('MurderRate', inplace=True)
cumsum = state.Population.cumsum()
cutoff = state.Population.sum() / 2.0
median_weight = state.MurderRate[cumsum >= cutoff].iloc[0]
median_weight

4.4

A média e a mediana ponderada são praticamente iguais